# Train Pixel Level Annotation Model

### This notebook uses pixel level annotations to train a Random Forest Classifier to predict labels

We assume Pixel level annotations are available, as produced by the "../data/annotations/transform_polygon_annotations_to_pixels.ipynb" notebook. Feature selection and grid_search for optimal parameters has been done in a separate notebook ("Coepelduynen/make_train_model_on_annotations_coepelduynen.ipynb") and those outcomes are taken as given in this notebook.

Change the set Variables cell below as desired and then run the entire notebook to get cross_validation results as well as a final model trained on all data.

Date: 2024-01-12\
Author: Pieter Kouyzer

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import pprint

from training.train import train_imbalanced_model, cross_validation_balance_on_date
from training.utils import get_cross_validation_results_filepath, get_model_filepath
from training.metric_calculation import calculate_average_metrics, get_metrics

In [ ]:
# Set Variables
location = "Voornes Duin"
satellite_constellation = "SuperView"
annotated_scaled_pixels_filepath = "C:/Users/pzhadmin/Data/remote-sensing/annotations/Voornes Duin_gecorrigeerd_100124_3_labels_pixel_annotations_scaled.parquet"


In [ ]:
# Optimal parameters and features
selected_features = ['b','i','ndvi','height']
optimal_parameters = {
    "n_estimators": 10, 
    "min_samples_split": 5, 
    "min_samples_leaf": 1,
    "max_features": 'auto', 
    "bootstrap": False
}

In [ ]:
df_scaled = pd.read_parquet(annotated_scaled_pixels_filepath)
df_scaled

In [ ]:
# This is to give an indication of the amount of data points per image
df_scaled['date'].value_counts()

### Cross Validation

We do cross-validation, where the folds are decided by the 'date' column. This is to avoid pixels from the same image from ending up in both the train and test datasets. We display the metrics averaged over the folds and write the results to a pickle.

In [ ]:
model = RandomForestClassifier(**optimal_parameters)

In [ ]:
results = cross_validation_balance_on_date(data=df_scaled, model=model, cv=4, features=selected_features, random_state=1337, sampling_type_boundary=100000)

In [ ]:
calculate_average_metrics(results=results)

In [ ]:
cross_validation_results_filepath = get_cross_validation_results_filepath(location=location, satellite_constellation=satellite_constellation, df=df_scaled)
print(f"Saving to {cross_validation_results_filepath}")
with open(cross_validation_results_filepath, "wb") as file:
    pickle.dump(results, file)

### Export Definitive model.

Trains a Random Forest Classifier model on all data and writes it to a pickle file for later use. This is the definitive model output by this notebook.

In [ ]:
final_model = RandomForestClassifier(**optimal_parameters)
train_imbalanced_model(
    X_train=df_scaled[selected_features], 
    y_train=df_scaled["label"], 
    model=final_model, 
    random_state=42, 
    sampling_type_boundary=100000
)
pprint.pprint(get_metrics(y=df_scaled["label"], X=df_scaled[selected_features], model=final_model))

In [ ]:
final_model_filepath = get_model_filepath(location=location, satellite_constellation=satellite_constellation, df=df_scaled)
print(f"Saving to {final_model_filepath}")
with open(final_model_filepath, "wb") as file:
    pickle.dump(final_model, file)